# Covid19 Data Analysis

#### Overview
...

#### Data Sources
* **Covid19**
    * [Johns Hopkins University - Covid19 Data](https://github.com/CSSEGISandData/COVID-19)
* **Demographics**
    * [US Census Bureau - 2019 Annual Social and Economic Supplements](https://www.census.gov/content/census/en/data/datasets/2019/demo/cps/cps-asec-2019.html)
    * [US Census Bureau - 2019 Current Population Survey](https://www.census.gov/content/census/en/data/datasets/2019/demo/cps/cps-basic-2019.html)
    * [US Census Bureau - International Demographic Overview](https://www.census.gov/data-tools/demo/idb/region.php?T=13&RT=0&A=both&Y=2020&C=&R=1)
    * [The World Bank - World Development Indicators](https://datacatalog.worldbank.org/dataset/world-development-indicators)
    * [The World Bank - Population Estimates and Projections](https://datacatalog.worldbank.org/dataset/population-estimates-and-projections)
    * [IMF - World Economic Outlook](https://www.imf.org/external/pubs/ft/weo/2020/01/weodata/download.aspx)

In [109]:
import logging
import datetime
import itertools
import subprocess
import os
from io import StringIO
import re
import sys
import json
import numpy as np
import pandas as pd
import pyarrow
import requests as rq

In [118]:
def CONFIG(env):
    """"""
    pid = 'analysis-covid19'
    now = datetime.datetime.now()
    cwd = os.getcwd()
    f_in = os.path.join(cwd,'in')
    f_out = os.path.join(cwd,'out')
    cov = os.path.join(f_in,'covid')
    
    cfg = {
        'paths': {
            'covid': cov
            ,'f_in': f_in
            ,'f_out': f_out
            ,'eda': os.path.join(cwd,'eda')
            ,'pq': os.path.join(f_out)
        }
        ,'jhu-refresh': os.path.join(cwd,'jhu-refresh.sh')
        ,'jhu-dly': { # johns hopkins university - global daily
            'path': os.path.join(cov,'jhu','csse-dly','{DATE}.csv')
            ,'dates': {
                'start': datetime.date(2020,4,12)
                ,'end': datetime.date(2020,9,30)
            }
        }
        ,'jhu-dly-us': { # johns hopkins university - us daily
            'path': os.path.join(cov,'jhu','csse-dly-us','{DATE}.csv')
            ,'dates': {
                'start': datetime.date(2020,4,12)
                ,'end': datetime.date(2020,9,30)
            }
        }
        ,'jhu-ts': { # johns hopkins university - timeseries
            'base': os.path.join(cov,'jhu','csse-ts','time_series_covid19_{STATUS}_{REGION}.csv')
            ,'options': {
                'status': ['confirmed','deaths','recovered']
                ,'region': ['global','US']
            }
            ,'errata': os.path.join(cov,'jhu','csse-ts','Errata.csv')
        }
        ,'jhu-who-ts': { # johns hopkins university - WHO timeseries
            'path': os.path.join(cov,'jhu','who-ts','who_covid_19_sit_rep_time_series.csv')
        }
        ,'jhu-fips': { # johns hopkins university - fips lkup table
            'path': os.path.join(cov,'jhu','csse-fips-lkup.csv')
        }
        ,'cb-acs': { # census bureau - american community survey
            'base': 'https://api.census.gov/data/{YEAR}/pep/charage?get={FIELDS}&for=state:{STATES}'
            ,'query': {
                'years': ['2019']
                ,'fields': ','.join(['POP','NAME'])
                ,'states': ','.join(['*'])
            }
        }
        ,'cb-idb': { # census bureau - international database
            'base': 'https://api.census.gov/data/timeseries/idb/1year?time={YEAR}&get={FIELDS}'
            ,'query': {
                'years': ['2019']
                ,'fields': ['AREA_KM2','NAME','AGE','POP','FIPS','SEX']
            }
        }
#         ,'wb-dev': {
#             'base'
#         }
#         ,'wb-pop': {
            
#         }
#         ,'imf-econ': {
            
#         }
    }
    
    return cfg

In [119]:
def LOGGER(env):
    """"""
    fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    formatter = logging.Formatter(fmt)
    logger = logging.getLogger(__name__)

    if env=='dev':
        level = logging.DEBUG
        feed = sys.stdout
    elif env=='qa':
        level = logging.INFO
        feed = StringIO()
    elif env=='prod':
        level = logging.WARNING
        feed = StringIO()
    else:
        raise Exception('Unknown environment.')

    handler = logging.StreamHandler(feed)
    handler.setFormatter(formatter)
    handler.setLevel(level)
    logger.addHandler(handler)
    logger.setLevel(level)

    return logger,feed


In [133]:
def JHU_DLY(base,start,end):
    """"""
    logger = logging.getLogger(__name__)
    
    file_dt = '%m-%d-%Y'
    data_dt = '%Y-%m-%d'
    
    df = pd.DataFrame(None)
    errs = []
    
    for i in range((end-start).days+1):
        dt = (start+datetime.timedelta(days=i))
        try:
            if df.empty:
                df = pd.read_csv(base.format(DATE=dt.strftime(file_dt)),header=0).assign(DATA_DT=dt.strftime(data_dt))
            else:
                df = df.append(pd.read_csv(base.format(DATE=dt.strftime(file_dt)),header=0).assign(DATA_DT=dt.strftime(data_dt)))
        except Exception as e:
            errs.append((dt,str(e)))
   
    df = CLEANER(df)
                               
    return df,errs

In [121]:
def CB_ACS(base,years,fields,states):
    """"""
    logger = logging.getLogger(__name__)
    
    df = pd.DataFrame(None)
    errs = []
    
    for yr in years:
        url = base.format(YEAR=yr,FIELDS=fields,STATES=states)
        response = rq.get(url)
        if response.status_code == requests.codes.ok:
            data = response.json()
            if df.empty:
                df = pd.DataFrame(data[1:],columns=data[0])
            else:
                df.append(pd.DataFrame(data[1:],columns=data[0]))
        else:
            errs.append(url)
    
    df = CLEANER(df)
    
    return df,errs

In [122]:
def DB_IDB(base,years,fields):
    """"""
    logger = logging.getLogger(__name__)
    
    df = pd.DataFrame(None)
    errs = []
    
    for yr in years:
        url = base.format(YEAR=yr,FIELDS=','.join(fields))
        response = rq.get(url)
        if response.status_code == requests.codes.ok:
            data = response.json()
            if df.empty:
                df = pd.DataFrame(data[1:],columns=data[0])
            else:
                df.append(pd.DataFrame(data[1:],columns=data[0]))
        else:
            errs.append(url)
    
    df = CLEANER(df)
    
    return df,errs

In [123]:
def CLEANER(df):
    """"""
    def COLUMN(x):
        if not x[0].isalnum():
            x=x[1:]
        if not x[-1].isalnum():
            x=x[:-1]
        return x.upper()

    df.columns = map(lambda x: COLUMN(x), df.columns)
    
    return df

In [129]:
def EDA(df,f_out,n=100):
    """"""
    logger = logging.getLogger(__name__)
    
    name = os.path.basename(f_out).split('.')[0]
    lb = '\n'
    lblb = '\n\n'
    
    # columns & types
    content = '# EDA - {} Files {}'.format(os.path.basename(name).upper(),lblb)
    content+='#### Column Name [IDX] -  Dtype (Head / Tail) \n'
    dtypes = df.dtypes.to_dict()
    head = df.head(1).T.iloc[:,0].to_list() # to_dict() - head.get(j)
    tail = df.tail(1).T.iloc[:,0].to_list() # to_dict() - tail.get(j)
    for i,j in enumerate(df.columns):
        content+='- **{}** [{}] - {} ({} / {}) {}'.format(j, i, dtypes.get(j), head[i], tail[i], lb)
    
    # html
    content+='{}#### Head / Tail [n={}] Sample {}'.format(lb+lblb,n,lblb)
    content+=(df.head(n).append(df.tail(n)).to_html(None,index=True,header=True))
    
    with open(f_out,'w') as f:
        f.write(content)

In [124]:
def PARQUET(df,f_out):
    """"""
    logger = logging.getLogger(__name__)
    df.to_parquet(f_out,engine='pyarrow',index=False,compression='gzip')

In [134]:
if __name__=='__main__':
# def MAIN():
#     """"""
    env = 'dev'
    LOG,FEED = LOGGER(env)
    CFG = CONFIG(env)
    
    sample_size = 20
    refresh = False
    if refresh:
        out = subprocess.run(['sh', CFG['jhu-refresh']]).returncode
        if out == 0:
            pass
        elif out == 1:
            raise Exception('Error while refreshing covid19 data.')
        elif out == 127:
            raise Exception('File not found.')
        else:
            raise Exception('Unknown shell out code: {}'.format(out))
        
            
    try:
        segment = 'covid'
        src = 'jhu'
        covid = {
            'jhu-dly': JHU_DLY(CFG['jhu-dly']['path'],**CFG['jhu-dly']['dates'])[0],
            'jhu-dly-us': JHU_DLY(CFG['jhu-dly-us']['path'],**CFG['jhu-dly-us']['dates'])[0],
            'jhu-ts': pd.read_csv(CFG['jhu-ts']['base'].format(STATUS='confirmed',REGION='US'),header=0),
            'jhu-who-ts': pd.read_csv(CFG['jhu-who-ts']['path'],header=0),
            'jhu-ts-err': pd.read_csv(CFG['jhu-ts']['errata'],header=0),
            'jhu-fips': pd.read_csv(CFG['jhu-fips']['path'],header=0),
        }
        for fname,df in covid.items():
            # clean df
            df = CLEANER(df)
            # md eda file
            fpath = os.path.join(CFG['paths']['eda'],'{}-{}.md'.format(segment,fname))
            EDA(df,fpath,n=sample_size)
            # parquet clean file
            fpath = os.path.join(CFG['paths']['pq'],segment,'{}-{}.parquet.gzip'.format(segment,fname))
            PARQUET(df,fpath)
    
    except Exception as e:
        LOG.critical(str(e))
        raise
    
    try:
        segment = 'demo'
        for fname in ['cb-acs','cb-idb']:
            if fname == 'cb-acs':
                df,errs = CB_ACS(CFG[fname]['base'],**CFG[fname]['query'])
            elif name == 'cb-idb':
                df,errs = DB_IDB(CFG[fname]['base'],**CFG[fname]['query'])
            else:
                raise Exception('Unknown file name.')
            # md eda file
            fpath = os.path.join(CFG['paths']['eda'],'{}-{}.md'.format(segment,fname))
            EDA(df,fpath,n=sample_size)
            # parquet clean file
            fpath = os.path.join(CFG['paths']['pq'],segment,'{}-{}.parquet.gzip'.format(segment,fname))
            PARQUET(df,fpath)
    
    except Exception as e:
        LOG.critical(str(e))
        raise